# Part 1

You find a program trying to generate some art. It uses a strange process that involves repeatedly enhancing the detail of an image through a set of rules.

The image consists of a two-dimensional square grid of pixels that are either on (`#`) or off (`.`). The program always begins with this pattern:

```
.#.
..#
###
```

Because the pattern is both 3 pixels wide and 3 pixels tall, it is said to have a size of 3.

Then, the program repeats the following process:

- If the size is evenly divisible by 2, break the pixels up into 2x2 squares, and convert each 2x2 square into a 3x3 square by following the corresponding enhancement rule.
- Otherwise, the size is evenly divisible by 3; break the pixels up into 3x3 squares, and convert each 3x3 square into a 4x4 square by following the corresponding enhancement rule.

Because each square of pixels is replaced by a larger one, the image gains pixels and so its size increases.

The artist's book of enhancement rules is nearby (your puzzle input); however, it seems to be missing rules. The artist explains that sometimes, one must rotate or flip the input pattern to find a match. (Never rotate or flip the output pattern, though.) Each pattern is written concisely: rows are listed as single units, ordered top-down, and separated by slashes. For example, the following rules correspond to the adjacent patterns:

```
../.#  =  ..
          .#

                .#.
.#./..#/###  =  ..#
                ###

                        #..#
#..#/..../#..#/.##.  =  ....
                        #..#
                        .##.
```

When searching for a rule to use, rotate and flip the pattern as necessary. For example, all of the following patterns match the same rule:

```
.#.   .#.   #..   ###
..#   #..   #.#   ..#
###   ###   ##.   .#.
```

Suppose the book contained the following two rules:

```
../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#
```

As before, the program begins with this pattern:

```
.#.
..#
###
```

The size of the grid (3) is not divisible by 2, but it is divisible by 3. It divides evenly into a single square; the square matches the second rule, which produces:

```
#..#
....
....
#..#
```

The size of this enhanced grid (4) is evenly divisible by 2, so that rule is used. It divides evenly into four squares:

```
#.|.#
..|..
--+--
..|..
#.|.#
```

Each of these squares matches the same rule (`../.# => ##./#../...`), three of which require some flipping and rotation to line up with the rule. The output for the rule is the same in all four cases:

```
##.|##.
#..|#..
...|...
---+---
##.|##.
#..|#..
...|...
```

Finally, the squares are joined into a new grid:

```
##.##.
#..#..
......
##.##.
#..#..
......
```

Thus, after 2 iterations, the grid contains 12 pixels that are on.

How many pixels stay on after 5 iterations?

In [52]:
from io import StringIO
from itertools import product
from textwrap import dedent

import numpy as np
import numpy.testing as npt

In [90]:
def arrayize(text: str) -> np.uint8:
        return np.uint8([[1 if c == '#' else 0 for c in r] for r in text.strip().split('/')])
    

def flips(arr: np.array) -> np.array:
    # original array
    yield arr
    # flip vertically
    yield np.flipud(arr)
    # flip horizontally
    yield np.fliplr(arr)
    # flip both
    yield np.flipud(np.fliplr(arr))
    
    
def transforms(arr: np.array) -> np.array:
    """Flip and rotate a 2D input array"""
    for farr in flips(arr):
        yield farr
        # transpose
        yield farr.T
        # rotate counter-clockwise
        yield np.rot90(arr)
        # rotate clockwise
        yield np.rot90(arr, k=-1)
        # rotate 180 degrees
        yield np.rot90(arr, k=2)
    
    
def load_rules(rows) -> dict:
    rules = {}
    
    for row in rows:
        from_, to = (arrayize(part) for part in row.split(' => '))
        for form in transforms(from_):
            rules[form.tobytes()] = to
    
    return rules

In [21]:
START_ARR = arrayize('.#./..#/###')
START_ARR

array([[0, 1, 0],
       [0, 0, 1],
       [1, 1, 1]], dtype=uint8)

In [91]:
def _test_input():
    return load_rules(StringIO(dedent("""\
        ../.# => ##./#../...
        .#./..#/### => #..#/..../..../#..#
        """)))
TEST_INPUT = _test_input()

In [92]:
def _puzzle_input():
    with open('./inputs/day21/input.txt') as f:
        return load_rules(f)
PUZZLE_INPUT = _puzzle_input()

In [93]:
len(PUZZLE_INPUT)

528

In [94]:
def grow(arr: np.array, rules: dict, iterations: int) -> np.array:
    for _ in range(iterations):
        if arr.shape[0] % 2 == 0:
            box_size = 2
        else:
            box_size = 3

        new_box_size = box_size + 1
        new_size = new_box_size * (arr.shape[0] // box_size)
        new_arr = np.empty((new_size, new_size), dtype=np.uint8)

        for (i, j), (ni, nj) in zip(
                product(range(0, arr.shape[0], box_size), repeat=2),
                product(range(0, new_size, new_box_size), repeat=2)):
            new_arr[ni:ni + new_box_size, nj:nj + new_box_size] = \
                rules[arr[i:i + box_size, j:j + box_size].tobytes()]
        
        arr = new_arr
    
    return arr

In [95]:
result = grow(START_ARR, TEST_INPUT, 2)
npt.assert_array_equal(
    result,
    np.uint8([
        [1, 1, 0, 1, 1, 0],
        [1, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 1, 0],
        [1, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ]))
assert result.sum() == 12

In [96]:
grow(START_ARR, PUZZLE_INPUT, 5).sum()

167

# Part 2

How many pixels stay on after 18 iterations?

In [97]:
grow(START_ARR, PUZZLE_INPUT, 18).sum()

2425195